In [83]:
import json
import networkx as nx
import pandas as pd
import operator
from pyvis.network import Network
import community as community_louvain
from infomap import Infomap


# JSON 파일을 딕셔너리로 읽어오는 함수
def load_dict_from_json(filename):
    with open(filename, 'r') as json_file:
        dictionary = json.load(json_file)
    return dictionary

def create_biblimetric_graph(dict) :
    G = nx.DiGraph()

    # 노드 추가
    nodes = list(dict.keys())

    G.add_nodes_from(nodes)
    edges = []
    personalization = {}
    for k in dict :
        for c in dict[k] :
            if c != 'self':
                try:
                    new_edge = (k, c, round(dict[k][c]/dict[k]['self'], 4))
                except:
                    new_edge = (k, c, 0)
                edges.append(new_edge)
            else :
                personalization[k] = dict[k][c]

    G.add_weighted_edges_from(edges)
    return G

In [111]:
cowork = load_dict_from_json("./number_cocite.json")
color_type = " "

G = create_biblimetric_graph(cowork)
pagerank = nx.pagerank(G)
infomap = Infomap()

node_to_index = {node: i for i, node in enumerate(G.nodes())}
index_to_node = {i: node for node, i in node_to_index.items()}

# Infomap 객체 생성
infomap = Infomap()

# NetworkX 그래프의 엣지를 Infomap 객체에 추가
for u, v, d in G.edges(data=True):
    infomap.add_link(node_to_index[u], node_to_index[v], d['weight'])

# 커뮤니티 탐지 수행
infomap.run()

# 결과 추출 및 원래 노드 이름으로 변환
communities = infomap.get_modules()
communities = {index_to_node[k]: v for k, v in communities.items()}

nodes_to_remove = [node for node, pr in pagerank.items() if pr < 0.005]
G.remove_nodes_from(nodes_to_remove)


net = Network(notebook=True, directed=True)

# NetworkX 그래프를 pyvis 네트워크로 변환
net.from_nx(G)

color_map = {
    "United States": "#dd4b39",
    "China": "#FFFF00",
    "France": "#20B2AA",
    "United Kingdom": "#20B2AA",
    "Germany": "#20B2AA"
}

for node in G.nodes():
    net.get_node(node)['title'] = f'{node} (PageRank: {round(pagerank[node], 4)})'
    net.get_node(node)['size'] = max(pagerank[node]*200, 5)

    if color_type == "Community" :
        try :
            net.get_node(node)['group'] = communities[node]
        except :
            net.get_node(node)['group'] = 0

    else :
        if node in color_map :
            net.get_node(node)['color'] = color_map[node]
        else : 
            eus = cowork[node].get("United Kingdom", 0) + cowork[node].get("Germany", 0) + cowork[node].get("France", 0)
            max_inf = max(cowork[node].get("United States", 0), cowork[node].get("China", 0), eus)
            if max_inf == cowork[node].get("United States", 0) :
                color = color_map["United States"]
            elif max_inf == cowork[node].get("China", 0) :
                color = color_map["China"]
            else :
                color = color_map["France"]
            net.get_node(node)['color'] = color

for i in range(len(G.edges())) :
    nx_edge = list(G.edges(data=True))[i]
    if  nx_edge[2]['width'] > 0.1 :
        color = net.get_node(nx_edge[1])['color']
    else :
        color = "DCDCDC"
    net.get_edges()[i] = {'width': nx_edge[2]['width']*10, 'from': nx_edge[0], 'to': nx_edge[1], 'arrows': 'to', 'color': color, 'title' : nx_edge[0] + f"-({round(nx_edge[2]['width'], 4)})->" + nx_edge[1]}


  Infomap v2.7.1 starts at Warning: When  cdn_resources is 'local' jupyter notebook has issues displaying graphics on chrome/safari. Use cdn_resources='in_line' or cdn_resources='remote' if you have issues viewing graphics in a notebook.
2024-05-16 09:58:25
  -> Input network: 
  -> No file output!
  OpenMP 201511 detected with 128 threads...
  -> Ordinary network input, using the Map Equation for first order network flows
Calculating global network flow using flow model 'undirected'... 
  -> Using undirected links.
  => Sum node flow: 1, sum link flow: 1
Build internal network with 138 nodes and 3550 links...
  -> One-level codelength: 6.31010998

Trial 1/1 starting at 2024-05-16 09:58:25
Two-level compression: 1.1% 0.15% 
Partitioned to codelength 0.635616071 + 5.59365911 = 6.229275181 in 8 modules.
Super-level compression: to codelength 6.229275181 in 8 top modules.

Recursive sub-structure compression: 0% . Found 2 levels with codelength 6.229275181

=> Trial 1/1 finished in 0.0085

In [112]:
net.show_buttons(filter_=['physics'])
net.show("node_paper_edge_confidence.html")

node_paper_edge_confidence.html
